<a href="https://colab.research.google.com/github/venkatavinayvijjapu/mlops/blob/main/MLOPS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install "zenml[server]"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.1/6.1 MB 41.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.8/209.8 kB 26.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 89.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 593.7/593.7 kB 54.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 96.6/96.6 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 148.1/148.1 kB 21.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.4/95.4 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 525.6/525.6 kB 48.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━

In [2]:
!zenml integration install sklearn -y

NumExpr defaulting to 2 threads.
⠹ Installing integrations...


In [3]:
pip install pyparsing==2.4.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.3 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 2.4.7
    Uninstalling pyparsing-2.4.7:
      Successfully uninstalled pyparsing-2.4.7


In [4]:
import  IPython

In [8]:
IPython.Application.instance().kernel.do_shutdown(restart=True)

{'status': 'ok', 'restart': True}

In [1]:
NGROK_TOKEN='2ZJXIG9NA5QVtzcxtd6VMevGe16_4ErmFY9NvKqdEzEKuNqhr'

In [2]:
!rm -rf .zen
!zenml init

NumExpr defaulting to 2 threads.
Initializing the ZenML global configuration version to 0.50.0
⠋ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠙ Initializing ZenML repository at /content.
⠹ Initializing ZenML repository at /content.
⠸ Initializing ZenML repository at /content.
⠼ Initializing ZenML repository at /content.
⠴ Initializing ZenML repository at /content.
⠦ Initializing ZenML repository at /content.
⠧ Initializing ZenML repository at /content.
⠇ Initializing ZenML repository at /content.
⠏ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.
⠙ Ini

In [3]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [4]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from typing import Tuple

In [9]:
@step
def importer()->Tuple[
    Annotated[np.ndarray,"x_train"],
    Annotated[np.ndarray,"x_test"],
    Annotated[np.ndarray,"y_train"],
    Annotated[np.ndarray,"y_test"],]:
    digits=load_digits()
    data=digits.images.reshape((len(digits.images),-1))
    x_train,x_test,y_train,y_test=train_test_split(data,digits.target,test_size=0.2,shuffle=False)
    return x_train,x_test,y_train,y_test

@step
def svc_trainer(x_train:np.ndarray,y_train:np.ndarray)->ClassifierMixin:
  model=SVC(gamma=0.001)
  model.fit(x_train,y_train)
  return model

@step
def evaluator(x_test:np.ndarray,y_test:np.ndarray,model:ClassifierMixin)->float:
  acc=model.score(x_test,y_test)
  print(acc)
  return acc


In [10]:
from zenml import pipeline
@pipeline
def digits_pipeline():
  x_train,x_test,y_train,y_test=importer()
  model=svc_trainer(x_train=x_train,y_train=y_train)
  evaluator(x_test=x_test,y_test=y_test,model=model)

In [11]:
digits_svc_pipeline=digits_pipeline()

Initiating a new run for the pipeline: digits_pipeline.
Registered new version: (version 2).
Executing a new run.
Using user: default
Using stack: default
  orchestrator: default
  artifact_store: default
Step importer has started.
Step importer has finished in 2.148s.
Step svc_trainer has started.
Step svc_trainer has finished in 0.446s.
Step evaluator has started.
0.9583333333333334
Step evaluator has finished in 0.418s.
Run digits_pipeline-2023_12_11-04_38_23_228356 has finished in 3.426s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [19]:
NGROK_TOKEN = '2ZJXIG9NA5QVtzcxtd6VMevGe16_4ErmFY9NvKqdEzEKuNqhr'
from zenml.environment import Environment

def start_zenml(port=8237):
    if Environment.in_google_colab():
        from pyngrok import ngrok
        public_url = ngrok.connect(port)
        print(public_url)
        !zenml up --blocking --port {port}
    else:
        !zenml up --port {port}

start_zenml()


INFO:pyngrok.ngrok:Opening tunnel named: http-8237-ec59b529-8207-4d99-8406-6fa8fb5a0df4


Opening tunnel named: http-8237-ec59b529-8207-4d99-8406-6fa8fb5a0df4


INFO:pyngrok.process.ngrok:t=2023-12-11T04:47:57+0000 lvl=info msg="no configuration paths supplied"


t=2023-12-11T04:47:57+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2023-12-11T04:47:57+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2023-12-11T04:47:58+0000 lvl=info msg=end pg=/api/tunnels id=745f699aa551bfa1 status=201 dur=56.785485ms
NumExpr defaulting to 2 threads.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [8690]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)


INFO:pyngrok.process.ngrok:t=2023-12-11T04:49:50+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2023-12-11T04:49:50+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


INFO:pyngrok.process.ngrok:t=2023-12-11T04:49:50+0000 lvl=info msg="session closing" obj=tunnels.session err=nil


t=2023-12-11T04:49:50+0000 lvl=info msg="session closing" obj=tunnels.session err=nil
INFO:     Shutting down
INFO:     Finished server process [8690]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError

